# Finding our best-performing salespeople and products

**Total points**: 24 points

## Introduction

AdventureWorks is a company that sells outdoor sporting equipment. The company has many different locations and has been recording the sales of different locations on various products. I'm tasked with the question: *What are our best products and salespeople and how can we use this information to improve our overall performance?*

**Analytical Context.** 

1. Look at how product ratings and total sales are related
2. See how products sell in different subcategories (bikes, helmets, socks, etc.)
3. Calculate which salespeople have performed the best in 2014
4. See if total sales are correlated with commission percentage

Importing the libraries, the `sql` extension + database

In [ ]:
%load_ext sql
import pandas as pd

In [ ]:
%sql sqlite:///AdventureWorks.db

In [ ]:
%%sql
SELECT * FROM product LIMIT 5;

 * sqlite:///AdventureWorks.db
Done.


productid,NAME,productnumber,makeflag,finishedgoodsflag,color,safetystocklevel,reorderpoint,standardcost,listprice,size,sizeunitmeasurecode,weightunitmeasurecode,weight,daystomanufacture,productline,class,style,productsubcategoryid,productmodelid,sellstartdate,sellenddate,discontinueddate,rowguid,modifieddate
1,Adjustable Race,AR-5381,f,f,,1000,750,0.0,0.0,,,,,0,,,,,,2008-04-30 00:00:00,,,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
2,Bearing Ball,BA-8327,f,f,,1000,750,0.0,0.0,,,,,0,,,,,,2008-04-30 00:00:00,,,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
3,BB Ball Bearing,BE-2349,t,f,,800,600,0.0,0.0,,,,,1,,,,,,2008-04-30 00:00:00,,,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
4,Headset Ball Bearings,BE-2908,f,f,,800,600,0.0,0.0,,,,,0,,,,,,2008-04-30 00:00:00,,,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
316,Blade,BL-2036,t,f,,800,600,0.0,0.0,,,,,1,,,,,,2008-04-30 00:00:00,,,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


## Finding our most popular products
the AWC Logo map is the most popular product sold in the company.

In [ ]:
%%sql
SELECT 
    pdc.productmodelid, 
    description,
    NAME,
    sum(orderqty) AS total_orders
    
FROM 
    ProductModelProductDescriptionCulture AS pdc
    
JOIN ProductDescription AS pd ON pdc.productdescriptionid = pd.productdescriptionid
JOIN Product AS p ON pdc.productmodelid = p.productmodelid
JOIN SalesOrderDetail AS sod ON p.productid = sod.productid

WHERE cultureid = 'en'
GROUP BY NAME
ORDER BY total_orders DESC
LIMIT 10

 * sqlite:///AdventureWorks.db
Done.


productmodelid,description,NAME,total_orders
2,Traditional style with a flip-up brim; one-size fits all.,AWC Logo Cap,8311
111,AWC logo water bottle - holds 30 oz; leak-proof.,Water Bottle - 30 oz.,6815
33,"Universal fit, well-vented, lightweight , snap-on visor.","Sport-100 Helmet, Blue",6743
11,Unisex long-sleeve AWC logo microfiber cycling jersey,"Long-Sleeve Logo Jersey, L",6592
33,"Universal fit, well-vented, lightweight , snap-on visor.","Sport-100 Helmet, Black",6532
33,"Universal fit, well-vented, lightweight , snap-on visor.","Sport-100 Helmet, Red",6266
1,"Light-weight, wind-resistant, packs to fit into a pocket.","Classic Vest, S",4247
114,"Includes 8 different size patches, glue and sandpaper.",Patch Kit/8 Patches,3865
32,"Short sleeve classic breathable jersey with superior moisture control, front zipper, and 3 back pockets.","Short-Sleeve Classic Jersey, XL",3864
11,Unisex long-sleeve AWC logo microfiber cycling jersey,"Long-Sleeve Logo Jersey, M",3636


### Looking at the correlation between quantity sold and price for each subcategory.

In [ ]:
%%sql 
WITH a AS (
    SELECT 
        productid,
        SUM(orderqty) AS quantity
    FROM 
        SalesOrderDetail
    GROUP BY productid
),

b AS (
    SELECT 
        p.productid,
        c.name AS category,
        sc.name AS subcategory,
        listprice
    FROM Product AS p
    JOIN ProductSubCategory as sc ON p.productsubcategoryid = sc.productsubcategoryid
    JOIN ProductCategory as c on sc.productcategoryid = c.productcategoryid
)

SELECT 
    category,
    subcategory,
    AVG(listprice) AS average_price_in_subcategory,
    SUM(quantity) AS total_items_sold_in_subcategory
    
FROM b
JOIN a ON b.productid = a.productid
GROUP BY subcategory
ORDER BY category ASC

 * sqlite:///AdventureWorks.db
Done.


category,subcategory,average_price_in_subcategory,total_items_sold_in_subcategory
Accessories,Bike Racks,120.0,3166
Accessories,Bike Stands,159.0,249
Accessories,Bottles and Cages,7.989999999999999,10552
Accessories,Cleaners,7.95,3319
Accessories,Fenders,21.98,2121
Accessories,Helmets,34.99,19541
Accessories,Hydration Packs,54.99,2761
Accessories,Locks,25.0,1087
Accessories,Pumps,19.99,1130
Accessories,Tires and Tubes,19.482727272727274,18006


There is positive correlation between average price and items sold ($\rho=0.68$). This is somewhat unexpected, since common sense tells us that the more expensive an item is, the lower the demand for it. It is possible that we are witnessing an instance of Simpson's Paradox here. To verify if that is indeed the case, we could instead compute the correlation coefficient for each subcategory, possibly evidencing a negative correlation coefficient in some subcategories. We will not do that right now, however, since it would make us deviate too much from our business problem.

## Finding our top salespeople

As mentioned earlier, we want to find our best salespeople and see whether or not we can incentivize them in an appropriate manner. Namely, we want to determine if the commission percentage we give them motivates them to make more and bigger sales.

### Top five performing salespeople by using the salesytd (Sales, year-to-date)

In [ ]:
%%sql
SELECT 
    businessentityid,
    salesytd
FROM SalesPerson
ORDER BY salesytd DESC
LIMIT 5

 * sqlite:///AdventureWorks.db
Done.


businessentityid,salesytd
276,4251368.5497
289,4116871.2277
275,3763178.1787
277,3189418.3662
290,3121616.3202


### Top five performing salespeople in 2014

In [ ]:
%%sql
SELECT 
    salespersonid,
    SUM(subtotal) AS totalsales
FROM salesorderheader 
WHERE orderdate >= '2014-01-01'
    AND salespersonid IS NOT NULL AND salespersonid <> ''
GROUP BY salespersonid
ORDER BY totalsales DESC
LIMIT 5

 * sqlite:///AdventureWorks.db
Done.


salespersonid,totalsales
289,1382996.5839000002
276,1271088.5216
275,1057247.3786
282,1044810.8277000003
277,1040093.4071


### Finding if there is a positive relationship between the total sales of the salespeople and their commission percentages

In [ ]:
%%sql
WITH a AS (
    SELECT
        salesorderid,
        SUM(orderqty*(unitprice*(1-unitpricediscount))) AS ordertotal
    FROM salesorderdetail
    GROUP BY salesorderid
),
b AS (
SELECT
    salespersonid,
    SUM(ordertotal) AS ordertotalsum
FROM a
JOIN salesorderheader AS soh ON a.salesorderid = soh.salesorderid
WHERE orderdate >= '2014-01-01'
    AND salespersonid IS NOT NULL
    AND salespersonid <> ""
GROUP BY salespersonid
ORDER BY ordertotalsum DESC
)

SELECT 
    b.*,
    commissionpct
FROM b
JOIN SalesPerson AS sp ON sp.businessentityid = b.salespersonid 

 * sqlite:///AdventureWorks.db
Done.


salespersonid,ordertotalsum,commissionpct
274,178584.36250800002,0.0
275,1057247.378572,0.012
276,1271088.5214610002,0.015
277,1040093.406901,0.015
278,435948.95516599994,0.01
279,787204.4288399998,0.01
280,504932.0440400001,0.01
281,777941.6520230001,0.01
282,1044810.8276870003,0.015
283,490466.31891500007,0.012


The correlation coefficient between `ordertotalsum` and `commissionpct` is $\rho=0.73$. This suggests that the salespeople who earn a high commission are also those who close the bigger deals.

## Grouping salespeople by currency since this company sells items worldwide

In [ ]:
%%sql
WITH a AS (
    SELECT
        salesorderid,
        SUM(orderqty*(unitprice*(1-unitpricediscount))) AS ordertotal
    FROM salesorderdetail
    GROUP BY salesorderid
),

b AS (
SELECT
    salespersonid,
    SUM(ordertotal) AS ordertotalsum
FROM a
JOIN salesorderheader AS soh ON a.salesorderid = soh.salesorderid
WHERE orderdate >= '2014-01-01'
    AND salespersonid IS NOT NULL
    AND salespersonid <> ""
GROUP BY salespersonid
ORDER BY ordertotalsum DESC
),

c AS (
    SELECT
        soh.salespersonid,
        soh.salesorderid,
        CASE
            WHEN tocurrencycode IS NULL THEN 'USD'
            ELSE tocurrencycode
        END AS tocurrencycode
    FROM SalesOrderHeader AS soh
    LEFT JOIN CurrencyRate as cr ON soh.currencyrateid = cr.currencyrateid
    WHERE orderdate >= '2014-00-00'
        AND soh.salespersonid IS NOT NULL
        AND soh.salespersonid <> ""
    ORDER BY salespersonid
)

SELECT DISTINCT
    b.*,
    commissionpct,
    c.tocurrencycode
FROM b
LEFT JOIN SalesPerson AS sp ON sp.businessentityid = b.salespersonid 
LEFT JOIN c ON b.salespersonid = c.salespersonid
ORDER BY tocurrencycode ASC, ordertotalsum DESC


 * sqlite:///AdventureWorks.db
Done.


salespersonid,ordertotalsum,commissionpct,tocurrencycode
286,585755.800528,0.018,AUD
285,21267.336,0.0,AUD
289,1382996.5839100003,0.02,CAD
278,435948.95516599994,0.01,CAD
274,178584.36250800002,0.0,CAD
290,867518.6139750001,0.016,EUR
288,581607.9321810001,0.018,EUR
287,56637.74780000001,0.0,EUR
282,1044810.8276870003,0.015,GBP
287,56637.74780000001,0.0,GBP
